In [26]:
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
from oandapyV20.exceptions import V20Error
import zmq
import pandas as pd
import tables 
import tstables  
import datetime
import configparser
import time
import utility_functions as uf
import threading
import os

try:
    config = configparser.ConfigParser()
    config.read('..\..\configinfo.cfg')

except:
    print( 'Error in reading configuration file' )

accountID = config['oanda_v20']['account_number_practice']
access_token = config['oanda_v20']['access_token_practice']
api = oandapyV20.API(access_token=access_token)

symbol = 'USD_TRY'
units = '1'

In [27]:
def market_order(symbol, units):

    order = { "order": {"type": "MARKET",
                                "instrument": symbol,
                                "units": units,
                                "timeInForce": "FOK",
                                "positionFill": "DEFAULT" } }
    
    return order      

In [50]:
def limit_order(symbol, units, entryprice, exitprice):

    order = { "order": { "type": "LIMIT", 
                         "instrument": symbol, 
                         "units": units, 
                         "price": entryprice, 
                         "stopLossOnFill": { "timeInForce": "GTC", "price": exitprice }, 
                         "timeInForce": "GTC", 
                         "positionFill": "DEFAULT" }}

    return order  

In [56]:
def limit_order(symbol, units, entryprice):

    order = { "order": { "type": "LIMIT", 
                         "instrument": symbol, 
                         "units": units, 
                         "price": entryprice, 
                         "timeInForce": "GTC", 
                         "positionFill": "DEFAULT" }}

    return order  

In [ ]:
def limit_order(symbol, units, entryprice, stoplossprice, takeprofitprice):

    order = { "order": { "type": "LIMIT", 
                         "instrument": symbol, 
                         "units": units, 
                         "price": entryprice, 
                         "stopLossOnFill": { "timeInForce": "GTC", "price": stoplossprice }, 
                         "takeProfitOnFill": { "timeInForce": "GTC", "price": takeprofitprice },
                         "timeInForce": "GTC", 
                         "positionFill": "DEFAULT" }}

    return order  

In [59]:
def stop_order(symbol, units, entryprice, stoplossprice, takeprofitprice):

    order = { "order": { "type": "STOP", 
                         "instrument": symbol, 
                         "units": units, 
                         "price": entryprice, 
                         "stopLossOnFill": { "timeInForce": "GTC", "price": stoplossprice }, 
                         "takeProfitOnFill": { "timeInForce": "GTC", "price": takeprofitprice },
                         "timeInForce": "GTC", 
                         "positionFill": "DEFAULT" }}

    return order  


In [52]:
#order = market_order('USD_TRY', '1')
order = limit_order('USD_TRY', '1', '3.4800', '3.5')
#order = limit_order('USD_TRY', '1', '3.4800')

r = orders.OrderCreate(accountID, data=order)
api.request(r)
print(r.response['orderCreateTransaction'])

{'type': 'LIMIT_ORDER', 'instrument': 'USD_TRY', 'units': '1', 'price': '3.48000', 'timeInForce': 'GTC', 'triggerCondition': 'DEFAULT', 'partialFill': 'DEFAULT', 'positionFill': 'DEFAULT', 'stopLossOnFill': {'price': '3.50000', 'timeInForce': 'GTC'}, 'reason': 'CLIENT_ORDER', 'id': '6145', 'userID': 4141160, 'accountID': '101-001-4141160-001', 'batchID': '6145', 'requestID': '42344526603160999', 'time': '2017-09-19T02:14:33.472804165Z'}


In [40]:
r = accounts.AccountDetails(accountID=accountID)
df = pd.DataFrame(api.request(r))

for e_position in df['account']['positions']:

    print( e_position )

{'instrument': 'EUR_USD', 'long': {'units': '0', 'pl': '-0.0005', 'resettablePL': '-0.0005', 'financing': '0.0000', 'unrealizedPL': '0.0000'}, 'short': {'units': '0', 'pl': '0.0000', 'resettablePL': '0.0000', 'financing': '0.0000', 'unrealizedPL': '0.0000'}, 'pl': '-0.0005', 'resettablePL': '-0.0005', 'financing': '0.0000', 'commission': '0.0000', 'unrealizedPL': '0.0000'}
{'instrument': 'USD_JPY', 'long': {'units': '0', 'pl': '-0.0252', 'resettablePL': '-0.0252', 'financing': '0.0000', 'unrealizedPL': '0.0000'}, 'short': {'units': '0', 'pl': '-0.0262', 'resettablePL': '-0.0262', 'financing': '0.0000', 'unrealizedPL': '0.0000'}, 'pl': '-0.0514', 'resettablePL': '-0.0514', 'financing': '0.0000', 'commission': '0.0000', 'unrealizedPL': '0.0000'}
{'instrument': 'USD_TRY', 'long': {'units': '0', 'pl': '-9.6206', 'resettablePL': '-9.6206', 'financing': '-0.0239', 'unrealizedPL': '0.0000'}, 'short': {'units': '0', 'pl': '-0.7886', 'resettablePL': '-0.7886', 'financing': '0.0028', 'unrealized

In [62]:
r = orders.OrderList(accountID)
api.request(r)
print(r.response)

{'orders': [{'id': '6161', 'createTime': '2017-09-19T02:20:51.117913958Z', 'type': 'LIMIT', 'instrument': 'USD_TRY', 'units': '1', 'timeInForce': 'GTC', 'price': '3.48000', 'triggerCondition': 'DEFAULT', 'partialFill': 'DEFAULT_FILL', 'positionFill': 'DEFAULT', 'state': 'PENDING'}], 'lastTransactionID': '6172'}


In [58]:
order = market_order('USD_TRY', '-2')
r = orders.OrderCreate(accountID, data=order)
api.request(r)

order = limit_order('USD_TRY', '1', '3.4800')
r = orders.OrderCreate(accountID, data=order)
api.request(r)

order = limit_order('USD_TRY', '1', '3.4850')
r = orders.OrderCreate(accountID, data=order)
api.request(r)


{'lastTransactionID': '6162',
 'orderCreateTransaction': {'accountID': '101-001-4141160-001',
  'batchID': '6162',
  'id': '6162',
  'instrument': 'USD_TRY',
  'partialFill': 'DEFAULT',
  'positionFill': 'DEFAULT',
  'price': '3.48500',
  'reason': 'CLIENT_ORDER',
  'requestID': '42344528193855614',
  'time': '2017-09-19T02:20:52.201144722Z',
  'timeInForce': 'GTC',
  'triggerCondition': 'DEFAULT',
  'type': 'LIMIT_ORDER',
  'units': '1',
  'userID': 4141160},
 'relatedTransactionIDs': ['6162']}

In [60]:
order = stop_order('EUR_USD', '100', '1.1970', '1.1960', '1.1980')

r = orders.OrderCreate(accountID, data=order)
api.request(r)

{'lastTransactionID': '6163',
 'orderCreateTransaction': {'accountID': '101-001-4141160-001',
  'batchID': '6163',
  'id': '6163',
  'instrument': 'EUR_USD',
  'partialFill': 'DEFAULT',
  'positionFill': 'DEFAULT',
  'price': '1.19700',
  'reason': 'CLIENT_ORDER',
  'requestID': '42344539836353381',
  'stopLossOnFill': {'price': '1.19600', 'timeInForce': 'GTC'},
  'takeProfitOnFill': {'price': '1.19800', 'timeInForce': 'GTC'},
  'time': '2017-09-19T03:07:08.983413546Z',
  'timeInForce': 'GTC',
  'triggerCondition': 'DEFAULT',
  'type': 'STOP_ORDER',
  'units': '100',
  'userID': 4141160},
 'relatedTransactionIDs': ['6163']}